# Logo Detection on Flickr27 Logo Dataset

In [1]:
import csv
import random
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import cv2
from keras.utils import np_utils
import glob

/home/mukesh/miniconda3/envs/statoil/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Dataset

In [2]:
txt_file = "flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt"
test_file = "flickr_logos_27_dataset/flickr_logos_27_dataset_query_set_annotation.txt"
image_dir = "flickr_logos_27_dataset/flickr_logos_27_dataset_images/"
extra_img = "more_images/"

In [3]:
annot_train = np.loadtxt(txt_file, dtype='a')
annot_test = np.loadtxt(test_file, dtype='a')
new_size=(224,224)

In [4]:
random.seed(42)
annot_train_sample=random.sample(list(annot_train),1000)
annot_test_sample=random.sample(list(annot_train),100)

# Processing Images
There is a lot of variation in size of images, all the images are resized to (224,224).

## challenges
If the images are resized then the co-ordinates of bounding needs to be adjusted, which is done by multiplying the ratio of new size to old size.

Test data has an extra class "none". So, I added some new images to training data which do not have logos from SUN 397 dataset with bounding box (0,0),(0,0).

In [5]:
def parse_annot(annot):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    subset_class = annot[2].decode('utf-8')
    xmin = int(annot[3].decode('utf-8'))
    ymin = int(annot[4].decode('utf-8'))
    xmax = int(annot[5].decode('utf-8'))
    ymax = int(annot[6].decode('utf-8'))
    return fn, class_name, subset_class, xmin, ymin, xmax, ymax

def parse_test_resize(annot,new_size=new_size):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    old_im=Image.open(image_dir+fn)
    old_size=old_im.size
    x_r = new_size[0]/old_size[0]
    y_r = new_size[1]/old_size[1]
    new_im = old_im.resize((new_size))
    return fn, new_im, class_name, x_r, y_r

def get_center_h_w(x1,y1,x2,y2):
    c_x=x1+int((x2-x1)/2)
    c_y=y1+int((y2-y1)/2)
    h=y2-y1
    w=x2-x1
    return c_x, c_y, w, h

def get_coords(c_x,c_y,w,h):
    x1 = int(c_x-(w/2))
    y1 = int(c_y-(h/2))
    x2 = int(c_x+(w/2))
    y2 = int(c_y+(h/2))
    return x1, y1, x2, y2

def process_img_resize(annot, new_size=new_size):
    fn, class_name, subset_class, xmin, ymin, xmax, ymax = parse_annot(annot)
    c_x, c_y, w, h = get_center_h_w(xmin, ymin, xmax, ymax)
    old_im=Image.open(image_dir+fn)
    old_size=old_im.size
    x_r = new_size[0]/old_size[0]
    y_r = new_size[1]/old_size[1]
    new_im = old_im.resize((new_size))
    c_x = int(c_x*x_r)
    c_y = int(c_y*y_r)
    w = int(w*x_r)
    h = int(h*y_r)
    return new_im, c_x, c_y, w, h, class_name

def new_img_to_train(images,new_size=new_size):
    img = Image.open(images)
    img = img.resize(new_size)
    class_name = 'none'
    c_x=0
    c_y=0
    h=0
    w=0
    return img,class_name, c_x, c_y, w, h

# Creating a Evaluation Metrics

Accuracy of bounding boxes cannot be calculated directly, It is calculated by another formula "Intersection over Union (IOU)".

In [6]:
#Evaluation Function
def IntersectionOverUnion(box1,box2):
    xmin1,ymin1,xmax1,ymax1 = box1[0],box1[1],box1[2],box1[3]
    xmin2,ymin2,xmax2,ymax2 = box2[0],box2[1],box2[2],box2[3]
    w_I = min(xmax1,xmax2)-max(xmin1,xmin2)
    h_I = min(ymax1,ymax2)-max(ymin1,ymin2)
    if w_I <=0 or h_I <=0:
        return 0
    I=w_I *h_I
    U=(abs(xmax1-xmin1)*abs(ymax1-ymin1))+(abs(xmax2-xmin2)*abs(ymax2-ymin2))-I
    return I/U

In [7]:
images=[]
labels=[]
box_cx=[]
box_cy=[]
box_w=[]
box_h=[]

for annot in annot_train:
    new_im, c_x, c_y, w, h, class_name = process_img_resize(annot)
    images.append(new_im)
    labels.append(class_name)
    box_cx.append(c_x)
    box_cy.append(c_y)
    box_w.append(w)
    box_h.append(h)

for fn in glob.glob(extra_img+'/*.jpg'):
    img,class_name, c_x, c_y, w, h=new_img_to_train(fn)
    images.append(img)
    labels.append(class_name)
    box_cx.append(c_x)
    box_cy.append(c_y)
    box_w.append(w)
    box_h.append(h)

In [8]:
file_name=[]
test_image=[]
test_labels=[]
x_shift=[]
y_shift=[]
for annot in annot_test_sample:
    fn, new_im, class_name, x_, y_ = parse_test_resize(annot)
    file_name.append(fn)
    test_image.append(new_im)
    test_labels.append(class_name)
    x_shift.append(x_)
    y_shift.append(y_)
    
def to_csv(predictions, pred_file):
    x1=[]
    y1=[]
    x2=[]
    y2=[]
    for out in predictions/10:
        xmin, ymin, xmax, ymax=get_coords(out[0],out[1],out[2],out[3])
        x1.append(xmin)
        y1.append(ymin)
        x2.append(xmax)
        y2.append(ymax)
    test_out=pd.DataFrame(data={'file':file_name,'x1':x1,'y1':y1,'x2':x2,'y2':y2})
    test_out.to_csv("results/"+pred_file+".csv", index=False)

In [9]:
data=pd.DataFrame(data={'image':images,'labels':labels,'c_x':box_cx,'c_y':box_cy,'w':box_w,'h':box_h})
data.head()

,c_x,c_y,h,image,labels,w
0,108,82,138,<PIL.Image.Image image mode=RGB size=224x224 a...,Adidas,156
1,146,160,73,<PIL.Image.Image image mode=RGB size=224x224 a...,Adidas,76
2,109,80,138,<PIL.Image.Image image mode=RGB size=224x224 a...,Adidas,163
3,112,106,103,<PIL.Image.Image image mode=RGB size=224x224 a...,Adidas,176
4,47,34,13,<PIL.Image.Image image mode=RGB size=224x224 a...,Adidas,21


In [10]:
test_data=pd.DataFrame(data={'image':test_image,'labels':test_labels,'x_shift':x_shift,'y_shift':y_shift})
test_data.head()

,image,labels,x_shift,y_shift
0,<PIL.Image.Image image mode=RGB size=224x224 a...,Cocacola,0.448000,0.957265
1,<PIL.Image.Image image mode=RGB size=224x224 a...,Yahoo,0.448000,3.500000
2,<PIL.Image.Image image mode=RGB size=224x224 a...,Intel,0.630986,0.558603
3,<PIL.Image.Image image mode=RGB size=224x224 a...,Ferrari,0.448000,0.494481
4,<PIL.Image.Image image mode=RGB size=224x224 a...,Cocacola,0.448000,0.524590


In [11]:
test_image = np.array([np.array(im) for im in test_data['image']])
test_image = test_image/255

In [12]:
images = np.array([np.array(im) for im in data['image']])
images = images/255

In [13]:
box=['c_x','c_y','w','h']
boxes = data[box].values

In [14]:
boxes.shape

(4616, 4)

# Create Model and Train 

In [15]:
from model import pre_trained_vgg16, pre_trained_mobilenet, vgg16_mobilenet, pre_trained_resnet50

In [16]:
def limit_mem():
    import keras.backend as K
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    print('gpu memory cleaned')

In [17]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, random_state=42, shuffle=True)
vggpredict_te=np.zeros((len(test_image), 4))
vggpredict=np.zeros((len(images), 4))
vggIOU=0
for train,test in folds.split(images,boxes):
    limit_mem()
    X_train, X_test = images[train], images[test]
    Y_train, Y_test = boxes[train], boxes[test]
    model=None
    model=pre_trained_vgg16(images)
    hist=model.fit(X_train,Y_train, validation_data=(X_test, Y_test),epochs=2, batch_size=4,verbose=1)
    preds=model.predict(X_test,batch_size=4, verbose=1)
    vggpredict_te +=model.predict(test_image,batch_size=4, verbose=1)
    vggpredict +=model.predict(images,batch_size=4, verbose=1)
    IOU=0
    for actual,pred in zip(Y_test,preds):
        IOU+=IntersectionOverUnion(actual,pred)
    print("IOU score on validation data for this fold =",IOU/len(X_test))
    vggIOU +=IOU/len(X_test) 
    del hist, IOU
print ("IOU score for this model on validation set :", vggIOU/10)

gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 327s 79ms/step - loss: 51.5416 - val_loss: 39.0036
Epoch 2/2
4616/4616 [==============================] - 103s 22ms/step
IOU score on validation data for this fold = 0.015026125078711013
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 319s 77ms/step - loss: 50.2331 - val_loss: 39.8109
Epoch 2/2
4616/4616 [==============================] - 149s 32ms/step
IOU score on validation data for this fold = 0.018261072354857476
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 437s 105ms/step - loss: 48.7506 - val_loss: 36.2799
Epoch 2/2
4616/4616 [==============================] - 88s 19ms/step
IOU score on validation data for this fold = 0.0033610064743302016
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2

In [18]:
to_csv(vggpredict_te,"vgg")

In [19]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, random_state=42, shuffle=True)
mobilenetpredict_te=np.zeros((len(test_image), 4))
mobilenetpredict=np.zeros((len(images), 4))
mobilenetIOU=0
for train,test in folds.split(images,boxes):
    limit_mem()
    X_train, X_test = images[train], images[test]
    Y_train, Y_test = boxes[train], boxes[test]
    model=None
    model=pre_trained_mobilenet(images)
    hist=model.fit(X_train,Y_train, validation_data=(X_test, Y_test),epochs=2, batch_size=4,verbose=1)
    preds=model.predict(X_test,batch_size=4, verbose=1)
    mobilenetpredict_te +=model.predict(test_image,batch_size=4, verbose=1)
    mobilenetpredict +=model.predict(images,batch_size=4, verbose=1)
    IOU=0
    for actual,pred in zip(Y_test,preds):
        IOU+=IntersectionOverUnion(actual,pred)
    print("IOU score on validation data for this fold =",IOU/len(X_test))
    mobilenetIOU +=IOU/len(X_test) 
    del hist, IOU
print ("IOU score for this model on validation set :", mobilenetIOU/10)

gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 112s 27ms/step - loss: 48.7650 - val_loss: 29.0037
Epoch 2/2
4616/4616 [==============================] - 44s 9ms/step
IOU score on validation data for this fold = 0.048169078548972566
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 118s 28ms/step - loss: 47.5961 - val_loss: 30.5324
Epoch 2/2
4616/4616 [==============================] - 44s 9ms/step
IOU score on validation data for this fold = 0.06163646020815252
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 119s 29ms/step - loss: 51.4109 - val_loss: 28.2090
Epoch 2/2
4616/4616 [==============================] - 48s 10ms/step
IOU score on validation data for this fold = 0.044316723703425584
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4

In [20]:
to_csv(mobilenetpredict_te,"mobilenet")

In [21]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, random_state=42, shuffle=True)
resnetpredict_te=np.zeros((len(test_image), 4))
resnetpredict=np.zeros((len(images), 4))
resnetIOU=0
for train,test in folds.split(images,boxes):
    limit_mem()
    X_train, X_test = images[train], images[test]
    Y_train, Y_test = boxes[train], boxes[test]
    model=None
    model=pre_trained_resnet50(images)
    hist=model.fit(X_train,Y_train, validation_data=(X_test, Y_test),epochs=2, batch_size=4,verbose=1)
    preds=model.predict(X_test,batch_size=4, verbose=1)
    resnetpredict_te +=model.predict(test_image,batch_size=4, verbose=1)
    resnetpredict +=model.predict(images,batch_size=4, verbose=1)
    IOU=0
    for actual,pred in zip(Y_test,preds):
        IOU+=IntersectionOverUnion(actual,pred)
    print("IOU score on validation data for this fold =",IOU/len(X_test))
    resnetIOU +=IOU/len(X_test) 
    del hist, IOU
print ("IOU score for this model on validation set :", resnetIOU/10)

gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 287s 69ms/step - loss: 47.0821 - val_loss: 32.9799
Epoch 2/2
4616/4616 [==============================] - 98s 21ms/step
IOU score on validation data for this fold = 0.03939682979967132
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 293s 70ms/step - loss: 46.3752 - val_loss: 29.0342
Epoch 2/2
4616/4616 [==============================] - 97s 21ms/step
IOU score on validation data for this fold = 0.06216115266369441
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 297s 71ms/step - loss: 48.1119 - val_loss: 35.2377
Epoch 2/2
4616/4616 [==============================] - 100s 22ms/step
IOU score on validation data for this fold = 0.05256206301593889
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/

In [22]:
to_csv(resnetpredict_te,"resnet")

In [23]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, random_state=42, shuffle=True)
vm_predict_te=np.zeros((len(test_image), 4))
vm_predict=np.zeros((len(images), 4))
vm_IOU=0
for train,test in folds.split(images,boxes):
    limit_mem()
    X_train, X_test = images[train], images[test]
    Y_train, Y_test = boxes[train], boxes[test]
    model=None
    model=vgg16_mobilenet(images)
    hist=model.fit(X_train,Y_train, validation_data=(X_test, Y_test),epochs=2, batch_size=4,verbose=1)
    preds=model.predict(X_test,batch_size=4, verbose=1)
    vm_predict_te +=model.predict(test_image,batch_size=4, verbose=1)
    vm_predict +=model.predict(images,batch_size=4, verbose=1)
    IOU=0
    for actual,pred in zip(Y_test,preds):
        IOU+=IntersectionOverUnion(actual,pred)
    print("IOU score on validation data for this fold =",IOU/len(X_test))
    vm_IOU +=IOU/len(X_test) 
    del hist, IOU
print ("IOU score for this model on validation set :", vm_IOU/10)

gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 415s 100ms/step - loss: 41.3309 - val_loss: 34.7828
Epoch 2/2
4616/4616 [==============================] - 178s 39ms/step
IOU score on validation data for this fold = 0.020964526328251947
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 420s 101ms/step - loss: 41.1543 - val_loss: 32.8700
Epoch 2/2
4616/4616 [==============================] - 177s 38ms/step
IOU score on validation data for this fold = 0.02664182701984555
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2
4154/4154 [==============================] - 420s 101ms/step - loss: 42.6663 - val_loss: 32.5234
Epoch 2/2
4616/4616 [==============================] - 183s 40ms/step
IOU score on validation data for this fold = 0.04364872291743626
gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/2

In [24]:
to_csv(vm_predict_te,"vgg_mobile")